# iMet Collection 2020 Kaggle Competition
Create a model that can assign attributes to artwork from Metropolitan Meuseum of Art.
Created by: Arnav Malhotra using fastai 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *

In [3]:
path = Path('data/'); path

WindowsPath('data')

In [ ]:
df = pd.read_csv(path/)